In [ ]:
# -*- coding: utf-8 -*-
"""
Created on:  06/22/22
author:      Ian Dwyer
institution: University of Colorado Denver

"""



""" ----------------------- IMPORT LIBRARIES ---------------------- """

##### 1. Import libraries


import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
import torch
import torch.nn as nn

""" --------------------------- GLOBALS --------------------------- """


#### 3. Define the model
#### defines the model as a custom nn.Module subclass

class MLP(nn.Module):                                                                 # must call nn.module subclass

    def __init__(self):
        super(MLP, self).__init__()                                               # super() calls parent class

        self.model = nn.Sequential(                                               # sequential() connect the layers
            nn.Linear(size, neurons),                                             # linear reg at each node
            nn.LeakyReLU(inplace=True),                                           # activation function
            nn.Linear(neurons, neurons*4),
            nn.LeakyReLU(inplace=True),                                           # activation function
            nn.Linear(neurons*4, neurons),
            nn.LeakyReLU(inplace=True),
            nn.Linear(neurons, outputs),
        )

    def forward(self, x):                                                         # pushes values through model
        x = self.model(x)

        return x

class CNN(nn.Module):

    def __init__(self):
      super(CNN,self).__init__()
                                                              # 3 convolutional layers
      self.conv1 = nn.Sequential(
            nn.Conv2d(3,32,kernel_size=3,padding=1),          #no loss in img dimension due to padding
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)   #1/4 of the feats go in, size passed to next layer 32*(224/2*224/2)=224*224*8
        )
      self.conv2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)   #1/4 of the feats go in, size passed to next layer 64*(224/4*224/4)=224*224*4
        )
      self.conv3 = nn.Sequential(
            nn.Conv2d(64,256,kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2) #1/4 of the feats go in,  size passed to next layer 256*(224/8*224/8)=224*224*4
        )

      self.fc = nn.Linear(256*28*28,outputs)        # 1 fully connected layer (256 28*28 images due to 3 layers of pooling)



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
                                                                                  #flatten before fully connecting convolutional layers
        x = x.view(x.size(0),-1)                                                  ##reshape in test and train is rendundant but not necessary to remove
        x = self.fc(x)
        return x


""" -------------------------- FUNCTIONS -------------------------- """

#### 4. Prepare the data

def CIFAR10_train():
    train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                   transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
                   ), batch_size=batch_size, shuffle=True)

    return train_loader

def CIFAR10_test():
    test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                   transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
                  ),batch_size=batch_size, shuffle=False)

    return test_loader

def transform(train_dir,test_dir,rotation,resize,norm_param):
  train_transforms = transforms.Compose([transforms.RandomRotation(rotation),
                                        transforms.RandomResizedCrop(resize),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize( norm_param[0],norm_param[1] )]
                                      )

  test_transforms = transforms.Compose([transforms.Resize(rotation),
                                        transforms.CenterCrop(resize),
                                        transforms.ToTensor(),
                                        transforms.Normalize( norm_param[0],norm_param[1])]
                                      )
  train_data = datasets.ImageFolder(train_dir,transform = train_transforms)
  test_data = datasets.ImageFolder(train_dir,transform = train_transforms)

  return train_data,test_data


#### 6. Start training

def train_MLP(train_loader,network,optimizer,criterion):
    for epoch in range(epochs):
        train_loss = 0
        train_acc = 0
        for data, target in train_loader:
            data = data.view(-1,size)                                               # reshapes to 1D tensor for input
            data, target = data.to(device), target.to(device)                       # changes both tensors to cuda
            logits = network(data)                                                  # inputs data into network, outputs tensor as 10 output predictions as logit
            loss = criterion(logits, target)                                        # cross entropy loss calculated (with softmax normalized probabilities) using prediction vs target
            optimizer.zero_grad()                                                   # clears gradient to avoid accumulating old gradient (loss) values when running optimizer
            loss.backward()                                                         # back propigates through network and updates new loss values
            optimizer.step()                                                        # single optimization step of gradients accumulated using stochiastic gradient descent for minima

            _,pred = logits.max(1)                                                  # Grab only predictions with value 1
            num_correct = (pred==target).sum().item()                               # find number of times pred outputs match target outputs, and sum the total items
            acc = num_correct / data.shape[0]                                       # accuracy calculation given current iteration over dataset size
            train_acc += acc                                                        # update accuracy of training data

            train_loss += loss.data                                                 # output loss found after backpropigation is accumulated

        print(
            'Train Epoch: {}  \nLoss: {:.6f} \nTraining Accuracy: {:.6f}'.format(
                epoch+1, train_loss/len(train_loader), train_acc/len(train_loader))
            )
        test_MLP(test_loader,network)                                             # prints test accuracy at that epoch


def train_CNN(train_loader,network,optimizer,criterion,stop):
    for epoch in range(epochs):
        train_loss = 0
        train_acc = 0
        test_acc = 0
        if epoch > 0:
          test_acc = test_CNN(test_loader,network)
          print('test accuracy: ', test_acc) ##prints previous test accuracy
          if test_acc >= stop:
            break
        for data, target in train_loader:
                                                                                    # reshapes to 1D tensor for input
            data, target = data.to(device), target.to(device)                       # changes both tensors to cuda
            logits = network(data)                                                  # inputs data into network, outputs tensor as 10 output predictions as logit
            loss = criterion(logits, target)                                        # cross entropy loss calculated (with softmax normalized probabilities) using prediction vs target
            optimizer.zero_grad()                                                   # clears gradient to avoid accumulating old gradient (loss) values when running optimizer
            loss.backward()                                                         # back propigates through network and updates new loss values
            optimizer.step()                                                        # single optimization step of gradients accumulated using stochiastic gradient descent for minima

            _,pred = logits.max(1)                                                  # Grab only predictions with value 1
            num_correct = (pred==target).sum().item()                               # find number of times pred outputs match target outputs, and sum the total items
            acc = num_correct / data.shape[0]                                       # accuracy calculation given current iteration over dataset size
            train_acc += acc                                                        # update accuracy of training data

            train_loss += loss.data                                                 # output loss found after backpropigation is accumulated



        print(
            '\nTrain Epoch: {}  \nLoss: {:.6f} \nTraining Accuracy: {:.6f}'.format(
                epoch+1, train_loss/len(train_loader), train_acc/len(train_loader))
            )


#### 7. Evaluate the model

def test_MLP(test_loader,network):
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, size)
        data, target = data.to(device), target.to(device)
        logits = network(data)

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    total_num = len(test_loader.dataset)
    acc = correct / total_num

    return acc

def test_CNN(test_loader,network):
    correct = 0
    network.eval()
    for data, target in test_loader:

        data, target = data.to(device), target.to(device)
        logits = network(data)

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    total_num = len(test_loader.dataset)
    acc = correct / total_num

    return acc

def CNN_avgtest(tests,test_loader,network):
  sum=0
  tests = 20
  for ii in range(tests):
    sum += test_CNN(test_loader,network)

  print('\nAverage test accuracy: ', sum/tests,'\n')


#### 8. Make predictions

def pred_CNN(test_loader, device):
    x, y = next(iter(test_loader))
    x, y = x.to(device), y.cuda()                                                   # note that .cuda() is original style and .to() is more dynamic
    out = network(x)                                                                # out is the network output of testing  values
    pred = out.argmax(dim=1)

    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std = [1/0.229, 1/0.224, 1/0.225]
    )
                                                  # due to softmax applied w cross entropy, the highest prob output is the prediction
    x = inv_normalize(x)

    return x,y,pred

def pred_MLP(test_loader, device):
    x, y = next(iter(test_loader))
    x = x.view(-1, size)
    x, y = x.to(device), y.cuda()                                                   # note that .cuda() is original style and .to() is more dynamic
    out = network(x)                                                                # out is the network output of testing  values
    pred = out.argmax(dim=1)
                                                   # due to softmax applied w cross entropy, the highest prob is the prediction

    return x,y,pred


def sample_MLP(channels,height,width,img, prediction, label):

    fig = plt.figure()
    for i in range(6):
        plt.imshow(np.transpose(img[i+10].reshape(channels,height,width),(1,2,0)))                   # required 3,32,32 shape then transposed to 32,32,3 to preserve orginal structure for plotting
        plt.title("Prediction = {} Label = {}".format(prediction[i].item(),
                    label[i].item())
                  )
        plt.show()


def sample_CNN(channels, width, height, img, prediction, label):

    fig = plt.figure()
    for i in range(6):
        plt.imshow(np.transpose(img[i].reshape(channels,width,height),(1,2,0)))                   # required 3,32,32 shape then transposed to 32,32,3 to preserve orginal structure for plotting
        plt.title("Prediction = {} Label = {}".format(prediction[i].item(),
                    label[i].item())
                  )
        plt.show()



""" --------------------------- PROGRAM --------------------------- """
### STICKING WITH leakyReLU for MLP DUE TO DUE TO LESS DOWNSIDES THAN TANH & SIG, and MORE RELIABLE THAN LEAKY
### ALSO ReLU IS FASTER THAN LOGISTIC REGRESSIONS LIKE TANH AND SIGMOID
### STOCHIASTIC GRADIENT DESCENT (SGD) IS USED IN CONFIGURE
### SGD WORKED BETTER THAN ADAM WITH AMSGRAD

#### BASIC SETTINGS
device = torch.device('cuda:0')                                                   # note that to.(device) is better than .cuda() due to dynamic input

img_width = 224
img_height = img_width
channels = 3

batch_size = 16               #80 imgs per flower (73 train, 7 test)    # small size used for CNN due to small validation set
learning_rate = .0001                                                              # step size. Lower step size for CNN w momentum helps
epochs = 300
outputs = 6
size = img_height*img_width*channels
neurons = size                                                                    # a neural network that is equal to or greater than the number of inputs 'should' be more accurate
momentum = 0.9

rotation = 30
resize = img_width
norm_param = (0.485,0.456,0.406),(0.229,0.224,0.225) # mean and standard deviation of normalized values of each channel (3,RGB)

stop_percent = 0.965 ##overrides epochs to get highest observed test score. Comment out if you dont know the tests potential


#### INIT DATA
train_dir = 'flowers6/train'
test_dir = 'flowers6/test'
train_data, test_data = transform(train_dir,test_dir,rotation,resize,norm_param)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle=True)

#### INIT MODEL PARAMETERS
network   = CNN().to(device)                                                 # MLP classifier class selected for neural network algorithm
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=0.9)                # stochastic gradient descent is selected for gradient analysis
criterion = nn.CrossEntropyLoss().to(device)                                 # cross entropy is selected for loss (aka cost) function

#### PRINT SETTINGS
print('\nSettings\nbatch_size: ',batch_size, '\nlearning_rate:',learning_rate, '\nmax epochs: ',epochs, '\nmomentum: ', momentum)

#### TRAIN MODEL
train_CNN(train_loader,network,optimizer,criterion,stop_percent)

#### TEST MODEL
tests=1
CNN_avgtest(tests,test_loader,network)

#### EVAL OF RANDOM INPUT
x,y,pred = pred_CNN(test_loader,device)
sample_CNN(channels, img_height, img_width, x.detach().cpu().numpy(), pred, y) #img, prediction, label



Settings
batch_size:  16 
learning_rate: 0.0001 
max epochs:  300 
momentum:  0.9

Train Epoch: 1  
Loss: 1.629074 
Training Accuracy: 0.537202
test accuracy:  0.5821917808219178

Train Epoch: 2  
Loss: 0.826294 
Training Accuracy: 0.715030
test accuracy:  0.730593607305936

Train Epoch: 3  
Loss: 0.783544 
Training Accuracy: 0.699405
test accuracy:  0.7602739726027398

Train Epoch: 4  
Loss: 0.760442 
Training Accuracy: 0.702381
test accuracy:  0.773972602739726

Train Epoch: 5  
Loss: 0.616206 
Training Accuracy: 0.773065
test accuracy:  0.7625570776255708

Train Epoch: 6  
Loss: 0.793439 
Training Accuracy: 0.732143
test accuracy:  0.7442922374429224

Train Epoch: 7  
Loss: 0.733317 
Training Accuracy: 0.723958
test accuracy:  0.7488584474885844

Train Epoch: 8  
Loss: 0.552504 
Training Accuracy: 0.790179
test accuracy:  0.8127853881278538

Train Epoch: 9  
Loss: 0.533620 
Training Accuracy: 0.787946
test accuracy:  0.821917808219178

Train Epoch: 10  
Loss: 0.555624 
Training Acc

In [ ]:
#for i, batch in enumerate(train_loader):
#    print(len(batch[0]))
batch = enumerate(train_loader)
print(batch)
#train_loader.item(0)
train_np = np.asarray(train_loader.dataset,dtype=object)
train_np.size
len(train_loader.dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  


438

12.514285714285714


In [3]:
!unzip flowers6.zip

Archive:  flowers6.zip
   creating: flowers6/
   creating: flowers6/test/
   creating: flowers6/test/buttercup/
  inflating: flowers6/test/buttercup/image_1194.jpg  
  inflating: flowers6/test/buttercup/image_1195.jpg  
  inflating: flowers6/test/buttercup/image_1196.jpg  
  inflating: flowers6/test/buttercup/image_1197.jpg  
  inflating: flowers6/test/buttercup/image_1198.jpg  
  inflating: flowers6/test/buttercup/image_1199.jpg  
  inflating: flowers6/test/buttercup/image_1200.jpg  
   creating: flowers6/test/daisy/
  inflating: flowers6/test/daisy/image_0874.jpg  
  inflating: flowers6/test/daisy/image_0875.jpg  
  inflating: flowers6/test/daisy/image_0876.jpg  
  inflating: flowers6/test/daisy/image_0877.jpg  
  inflating: flowers6/test/daisy/image_0878.jpg  
  inflating: flowers6/test/daisy/image_0879.jpg  
  inflating: flowers6/test/daisy/image_0880.jpg  
   creating: flowers6/test/fritillary/
  inflating: flowers6/test/fritillary/image_0714.jpg  
  inflating: flowers6/test/friti

In [ ]:
import numpy as np
def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std = [1/0.229, 1/0.224, 1/0.225])
    img = inv_normalize(img)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get random images
images, labels = iter(train_loader).next()


## get specific images
#images, labels = train_loader
#for i in range(images.shape[0]):
#images[i] = images[i]


# show images
imshow(images[0])


NameError: ignored